In [69]:
import torch
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

from newsapi import NewsApiClient
from datetime import datetime, timedelta
from transformers import pipeline
from warnings import filterwarnings

filterwarnings("ignore")


In [85]:
# Functions

def fetch_news_with_sentiment(company_name, api_key, language='en'):
    # Initialize News API client with your API key
    newsapi = NewsApiClient(api_key=api_key)

    # Define the query to search for news related to the company
    query = company_name

    # Fetch news articles related to the company for the last 'days' days
    news = newsapi.get_everything(q=query, language=language, sort_by='publishedAt')

    # Create a list comprehension to extract headlines
    headlines = [article['title'] for article in news['articles']]

    # Perform sentiment analysis
    sentiment_analyzer = pipeline("text-classification", model="ProsusAI/finbert")
    result = sentiment_analyzer(headlines)

    # Map labels to numeric values
    label_mapping = {'positive': 1, 'neutral': 0, 'negative': -1}
    sentiment_labels = [label_mapping[item['label']] for item in result]

    # Create a DataFrame
    df = pd.DataFrame({'Headline': headlines, 'Sentiment': sentiment_labels})

    # Convert the date string to a datetime object and extract only the date part
    articles_with_dates = [(article['publishedAt'], datetime.fromisoformat(article['publishedAt'][:-1]).date()) for article in news['articles']]

    # Set the date as index
    df['Date'] = [date for _, date in articles_with_dates]
    df.set_index('Date', inplace=True)


    return df


def plot_sentiment_histogram(df, company_name):
    # Define custom pastel colors for each sentiment category
    colors = {1: '#7fbf7b', 0: '#afafee', -1: '#ff8888'}

    # Plot histogram
    fig = px.histogram(df, x='Sentiment', color='Sentiment',
                       labels={'Sentiment': 'Sentiment', 'count': 'Count'},
                       title='Sentiment Distribution for ' + company_name,
                       category_orders={'Sentiment': [-1, 0, 1]},
                       color_discrete_map=colors,  # Assign custom colors
                       barmode='overlay',  # Overlay bars
                       )

    # Customize layout
    fig.update_layout(
        plot_bgcolor='black',  # Background color
        paper_bgcolor='black',  # Plot area color
        font=dict(color='white'),  # Font color
        legend=dict(font=dict(color='white')),  # Legend font color
        xaxis=dict(
            gridcolor='white',  # Grid color
            tickfont=dict(color='white'),  # X-axis tick color
        ),
        yaxis=dict(
            gridcolor='darkgray',  # Grid color
            tickfont=dict(color='white'),  # Y-axis tick color
        ),
        bargap=0.2,  # Gap between bars
        bargroupgap=0.1,  # Gap between groups of bars
    )

    # Show the plot
    fig.show()


In [87]:
# Example usage
company_name = "NVIDIA"
api_key = '381793b3d6834758918838bca0cf52ee'
df = fetch_news_with_sentiment(company_name, api_key)

In [88]:
df

,Headline,Sentiment
Date,,
2024-05-24,"Stock market today: Tech stocks lead S&P 500, ...",1
2024-05-24,U.S. stocks bounce back before long weekend,1
2024-05-24,Memorial Day Laptop Deal - Lenovo Ideapad For ...,0
2024-05-24,The Top 10 AI Stocks Hedge Funds Are Buying: G...,0
2024-05-24,This Slim Lenovo Gaming Laptop Has An OLED Dis...,0
...,...,...
2024-05-24,"Trending tickers: Nvidia, Boeing, National Gri...",0
2024-05-24,Nvidia's Massive FCF Margins Could Make it Wor...,1
2024-05-24,Nvidia CEO Jensen Huang’s net worth ballooned ...,1


In [91]:
plot_sentiment_histogram(df, company_name)